In [31]:
# example_plotting.ipynb
# Authors: Stephan Meighen-Berger
# Plot the output from example.py

In [2]:
# imports
import sys
sys.path.append('../')
from hebe import HEBE, config
import awkward as ak
import numpy as np
import h5py
import pyarrow as pa
import pyarrow.parquet as pq
import json

In [3]:
# Truth fetcher
def truth_fetcher(filename: str):
    """ Fetches the true values from the injection file

    Parameters
    ----------
    filename : str
        File to load

    Returns
    -------
    truth : dict
        Dictionary containig the injection information
    """
    LI_raw = h5py.File(
        filename
    )
    final_states = {}
    for elem in config['run']['data sets']:
        if elem[0]:
            final_states[elem[1]] = (
                LI_raw[config['run']['group name']][elem[1]]
            )
    print('Finished loading')
    # Creating data set
    print('Creating the data set for further propagation')
    truth = {}
    for key in final_states.keys():
        print('-------------------------------')
        print('Starting set')
        truth[key] = []
        for event in final_states[key]:
            # Making sure the event id is okay:
            if event[1] in config['particles']['explicit']:
                event_id = event[1]
            else:
                event_id = config['particles']['replacement']
            pos = np.array(event[2])
            # Introducing an injection offset caused by the different
            # coordinate systems between LI and the t2k file.
            pos = pos + np.array(config['detector']['injection offset'])
            truth[key].append({
                "time": 0.,
                "theta": event[3][0],
                "phi": event[3][1],
                # TODO: This needs to be removed once the coordinate
                # systems match!
                "pos": pos,
                "energy": event[4],
                "particle_id": event_id,
                'length': config['lepton propagator']['track length'],
                'event id': event_id
            })
    return truth

In [4]:
set_id = '18'
truth_dict = truth_fetcher('./output/data_' + set_id + '_output_LI.h5')

Finished loading
Creating the data set for further propagation
-------------------------------
Starting set
-------------------------------
Starting set


In [5]:
hebe = HEBE()

-------------------------------------------
Dumping config file
Finished dump
-------------------------------------------
-------------------------------------------
Setting up the detector
Starting up the detector handler
Finished the detector
-------------------------------------------
Setting up leptopn propagation
Using proposal
The proposal version is 7.2.1
This assumes a homogeneous medium!
Using new setup
----------------------------------------------------
Setting up proposal
Finished setup
----------------------------------------------------
Finished the lepton propagator
-------------------------------------------
-------------------------------------------
Setting up photon propagation
--------------------------------------------
Constructing the photon propagator
Using olympus
Setting up the medium
Finished the medium
-------------------------------------------------------
Setting up the photon generator
Finished the photon generator
----------------------------------------

In [6]:
results = ak.from_parquet(
    config['photon propagator']['storage location'] + 'rset_' + set_id + '_final_1' + '.parquet'
)

In [28]:
config['plot']['zrange'] = [-2500, -1300]
config['plot']['xrange'] = [-1500, 1500]
config['plot']['yrange'] = [-1500, 1500]
hit_set = []
minimal_hits = 10
for id_event, event in enumerate(results):
    total_hits = sum([len(dom) for dom in event])
    if total_hits > minimal_hits:
        hit_set.append([truth_dict['final_1'][id_event], total_hits, event])
max_event = np.argmax([event[1] for event in hit_set])
dom_hits_list = []
for event in hit_set:
    counter = 0
    for dom in event[2]:
        if len(dom) > 0:
            counter += 1
    dom_hits_list.append(counter)
max_dom = np.argmax(dom_hits_list)
print("Fraction of useful events: %f" % (len(hit_set) / len(results)))
print(max_event)
print(max_dom)

Fraction of useful events: 0.500000
2
4


In [29]:
print('Most hits')
id_event = max_event
hebe.event_plotting(
    hebe._det,
    hit_set[id_event][2]
)
print('Event ID')
print(id_event)
print('Hits')
print(sum([len(dom) for dom in hit_set[id_event][2]]))
print('Energy in GeV')
print(hit_set[id_event][0]['energy'])

Most hits


Event ID
2
Hits
3470
Energy in GeV
572.3372888034975


In [30]:
print('Most doms')
id_event = max_dom
hebe.event_plotting(
    hebe._det,
    hit_set[id_event][2]
)
print('Event ID')
print(id_event)
print('Hits')
print(sum([len(dom) for dom in hit_set[id_event][2]]))
print('Energy in GeV')
print(hit_set[id_event][0]['energy'])

Most doms


Event ID
4
Hits
86
Energy in GeV
205.15833518818528
